# Testing hexagonal kernels and backward gradients

In [1]:
import os
import sys
import numpy as np
import tensorflow as tf

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
tf.logging.set_verbosity(tf.logging.DEBUG)
sys.path.append(os.path.abspath('../'))

/home/paul/anaconda3/envs/tf_develop/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
ls

hex_layers_tests.ipynb  hex_rot_layers_tests.ipynb


# Hexagonal Rotation layers

Test out the basic hexagonal layers.

In [3]:
import tf_extended as tfx
from tensorflow.contrib import hex_layers

In [4]:
size = 6
shape = [size, size]
a = np.reshape(np.linspace(1.0, size**2, num=size**2, dtype=np.float32), shape)

print('Test Tensor: \n', a)
a = a.reshape([1, size, size, 1])

Test Tensor: 
 [[ 1.  2.  3.  4.  5.  6.]
 [ 7.  8.  9. 10. 11. 12.]
 [13. 14. 15. 16. 17. 18.]
 [19. 20. 21. 22. 23. 24.]
 [25. 26. 27. 28. 29. 30.]
 [31. 32. 33. 34. 35. 36.]]


In [5]:
t = hex_layers.hex_depthwise_conv2d(a, 3)
t = hex_layers.hex_downscale2d(a, 2)

In [6]:
# Test cartesian to hexagonal tiling.
t = hex_layers.hex_from_cartesian(a, downscale=False)
with tf.Session(config=tf.ConfigProto(log_device_placement=False)) as sess:
    print(t.eval().squeeze())

[[ 1.   2.   3.   4.   5.   6. ]
 [ 7.5  8.5  9.5 10.5 11.5 12. ]
 [13.  14.  15.  16.  17.  18. ]
 [19.5 20.5 21.5 22.5 23.5 24. ]
 [25.  26.  27.  28.  29.  30. ]
 [31.5 32.5 33.5 34.5 35.5 36. ]]


In [7]:
a.shape
t.get_shape()

TensorShape([Dimension(1), Dimension(6), Dimension(6), Dimension(1)])

# Forward hexagonal rotation depthwise convolution 2D

Testing the forward convolution.

In [9]:
size = 5
shape = [size, size]

# Input tensor
a = np.reshape(np.linspace(1.0, size**2, num=size**2, dtype=np.float32), shape)
print('Test Tensor: \n', a, '\n')
a = a.reshape([1, size, size, 1])

# Rotation tensor
ralpha_zero = np.zeros(shape, dtype=np.float32)
print('Rotation Tensor: \n', ralpha_zero)
ralpha_zero = ralpha_zero.reshape([1, size, size, 1])

Test Tensor: 
 [[ 1.  2.  3.  4.  5.]
 [ 6.  7.  8.  9. 10.]
 [11. 12. 13. 14. 15.]
 [16. 17. 18. 19. 20.]
 [21. 22. 23. 24. 25.]] 

Rotation Tensor: 
 [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


## No rotation

In [10]:
# Testing several simple configurations of kernels.
radius = 2
ksize = 2*radius+1
stride = 1

with tf.Session(config=tf.ConfigProto(log_device_placement=False)) as sess:
    idx = 0
    for rad in range(radius+1):
        rsize = max(1, rad*6)
        for i in range(rsize):
            
            # Basic kernel
            f = np.zeros(shape=[ksize**2], dtype=np.float32)
            f[idx] = 1.
            # Hexagonal rotation depthwise conv2d.
            f = np.reshape(f, [ksize, ksize, 1, 1])
            t = tf.nn.hex_rot_depthwise_conv2d_native(
                a, f, ralpha_zero, [1, stride, stride, 1], 'SAME', 'NHWC')
            r = t.eval()
            # Classic hexagonal dw conv2d.
            tc = tf.nn.hex_depthwise_conv2d(a, f, [1, stride, stride, 1], 'SAME', data_format='NHWC')
            rc = t.eval()
            # Equality between methods?
            req = np.all(rc == r)
            print('TEST kernel %i. Equality between kernels: %s' % (idx, req))
            if not req:
                print('FAILED result: \n', r.squeeze(), '\nClassic:\n', rc.squeeze(), '\n')
            idx += 1

TEST kernel 0. Equality between kernels: True
TEST kernel 1. Equality between kernels: True
TEST kernel 2. Equality between kernels: True
TEST kernel 3. Equality between kernels: True
TEST kernel 4. Equality between kernels: True
TEST kernel 5. Equality between kernels: True
TEST kernel 6. Equality between kernels: True
TEST kernel 7. Equality between kernels: True
TEST kernel 8. Equality between kernels: True
TEST kernel 9. Equality between kernels: True
TEST kernel 10. Equality between kernels: True
TEST kernel 11. Equality between kernels: True
TEST kernel 12. Equality between kernels: True
TEST kernel 13. Equality between kernels: True
TEST kernel 14. Equality between kernels: True
TEST kernel 15. Equality between kernels: True
TEST kernel 16. Equality between kernels: True
TEST kernel 17. Equality between kernels: True
TEST kernel 18. Equality between kernels: True


## Constant rotation tensor

In [11]:
# Rotation tensor
ralpha_drift = np.float32(0 / 6.) 
# ralpha_drift = np.float32(0)
ralpha = np.zeros(shape, dtype=np.float32) + ralpha_drift

print('Rotation Tensor: \n', ralpha)
ralpha = ralpha.reshape([1, size, size, 1])

Rotation Tensor: 
 [[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [12]:
# Testing several simple configurations of kernels.
radius = 2
ksize = 2*radius+1
stride = 1

with tf.Session(config=tf.ConfigProto(log_device_placement=False)) as sess:
    idx = 0
    for rad in range(radius+1):
        rsize = max(1, rad*6)
        for i in range(rsize):
            
            # Basic kernel
            f = np.zeros(shape=[ksize**2], dtype=np.float32)
            f[idx] = 1.
            # Hexagonal rotation depthwise conv2d.
            f = np.reshape(f, [ksize, ksize, 1, 1])
            t = tf.nn.hex_rot_depthwise_conv2d_native(
                a, f, ralpha, [1, stride, stride, 1], 'SAME', 'NHWC')
            r = t.eval()
            print('TEST kernel %i.' % (idx))
            print('TEST result: \n', r.squeeze(), '\n')
            idx += 1

TEST kernel 0.
TEST result: 
 [[ 1.  2.  3.  4.  5.]
 [ 6.  7.  8.  9. 10.]
 [11. 12. 13. 14. 15.]
 [16. 17. 18. 19. 20.]
 [21. 22. 23. 24. 25.]] 

TEST kernel 1.
TEST result: 
 [[ 0.  0.  0.  0.  0.]
 [ 1.  2.  3.  4.  5.]
 [ 0.  6.  7.  8.  9.]
 [11. 12. 13. 14. 15.]
 [ 0. 16. 17. 18. 19.]] 

TEST kernel 2.
TEST result: 
 [[ 0.  0.  0.  0.  0.]
 [ 2.  3.  4.  5.  0.]
 [ 6.  7.  8.  9. 10.]
 [12. 13. 14. 15.  0.]
 [16. 17. 18. 19. 20.]] 

TEST kernel 3.
TEST result: 
 [[ 2.  3.  4.  5.  0.]
 [ 7.  8.  9. 10.  0.]
 [12. 13. 14. 15.  0.]
 [17. 18. 19. 20.  0.]
 [22. 23. 24. 25.  0.]] 

TEST kernel 4.
TEST result: 
 [[ 6.  7.  8.  9. 10.]
 [12. 13. 14. 15.  0.]
 [16. 17. 18. 19. 20.]
 [22. 23. 24. 25.  0.]
 [ 0.  0.  0.  0.  0.]] 

TEST kernel 5.
TEST result: 
 [[ 0.  6.  7.  8.  9.]
 [11. 12. 13. 14. 15.]
 [ 0. 16. 17. 18. 19.]
 [21. 22. 23. 24. 25.]
 [ 0.  0.  0.  0.  0.]] 

TEST kernel 6.
TEST result: 
 [[ 0.  1.  2.  3.  4.]
 [ 0.  6.  7.  8.  9.]
 [ 0. 11. 12. 13. 14.]
 [ 0. 16. 17.

# Backprop Input - Hexagonal rotation depthwise convolution 2D

Testing the backward convolution, input gradient.

In [16]:
size = 3
shape = [size, size]
a = np.reshape(np.linspace(1.0, size**2, num=size**2, dtype=np.float32), shape)

print('Test Tensor: \n', a)
a = a.reshape([1, size, size, 1])

# Rotation tensor
ralpha_drift = np.float32(1. / 6.) 
ralpha = np.zeros(shape, dtype=np.float32) + ralpha_drift
print('Rotation Tensor: \n', ralpha)
ralpha = ralpha.reshape([1, size, size, 1])

Test Tensor: 
 [[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]
Rotation Tensor: 
 [[0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667]]


In [17]:
# Testing several simple configurations of kernels.
insize = 3
radius = 2
ksize = 2*radius+1
stride = 1

with tf.Session(config=tf.ConfigProto(log_device_placement=False)) as sess:
    for k in range(ksize**2):
        # Basic kernel
        f = np.zeros(shape=[ksize**2], dtype=np.float32)
        f[k] = 1.
        print('TEST kernel: \n', f)
        # Hex. depthwise conv2d
        f = np.reshape(f, [ksize, ksize, 1, 1])
        t = tf.nn.hex_rot_depthwise_conv2d_native_backprop_input(
            [1, insize, insize, 1], f, ralpha, a, [1, stride, stride, 1], 'SAME', data_format='NHWC')
        print('TEST result: \n', t.eval().squeeze(), '\n')


TEST kernel: 
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
TEST result: 
 [[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]] 

TEST kernel: 
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
TEST result: 
 [[2. 3. 0.]
 [5. 6. 0.]
 [8. 9. 0.]] 

TEST kernel: 
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
TEST result: 
 [[4. 5. 6.]
 [8. 9. 0.]
 [0. 0. 0.]] 

TEST kernel: 
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
TEST result: 
 [[0. 4. 5.]
 [7. 8. 9.]
 [0. 0. 0.]] 

TEST kernel: 
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
TEST result: 
 [[0. 1. 2.]
 [0. 4. 5.]
 [0. 7. 8.]] 

TEST kernel: 
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
TEST result: 
 [[0. 0. 0.]
 [1. 2. 3.]
 [0. 4. 5.]] 

TEST kernel: 
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.]
TEST result: 
 [[0. 0. 0.

In [ ]:
tf.nn.hex_rot_depthwise_conv2d_native_backprop_input?

## Backward hexagonal depthwise convolution 2D - kernel

Testing the backward convolution, input gradient.

In [ ]:
size = 3
shape = [size, size]
a = np.reshape(np.linspace(1.0, size**2, num=size**2, dtype=np.float32), shape)

print('Test Tensor: \n', a)
a = a.reshape([1, size, size, 1])

In [ ]:
# Testing several simple configurations of kernels.
ksize = 3

with tf.Session(config=tf.ConfigProto(log_device_placement=False)) as sess:
    for k in range(ksize**2):
        # Basic kernel
        f = np.zeros(shape=[ksize**2], dtype=np.float32)
        f[k] = 1.
        print('TEST out gradient: \n', f.reshape([3, 3]))
        # Hex. depthwise conv2d
        f = np.reshape(f, [1, ksize, ksize, 1])
        t = tf.nn.hex_depthwise_conv2d_native_backprop_filter(
            f, [3, 3, 1, 1], a, [1, 1, 1, 1], 'SAME', data_format='NHWC')        
        print('TEST result: \n', t.eval().squeeze().flatten(), '\n')


## Help...

In [ ]:
tf.nn.hex_depthwise_conv2d?

In [ ]:
tf.nn.depthwise_conv2d_native_backprop_filter?

In [ ]:
tf.nn.hex_depthwise_conv2d_native_backprop_input?

In [ ]:
tf.nn.hex_depthwise_conv2d_native_backprop_filter?